In [2]:
import duckdb

# 连接到持久化数据库（如果文件不存在，会自动创建）
con = duckdb.connect('database.duckdb')

con.execute("SET memory_limit = '4GB'")
con.execute("SET index_scan_percentage = 0.9")
con.execute("SET index_scan_max_count = 100")
con.execute("SET threads = 6")


In [9]:
con.execute("""
    CREATE TABLE my_table (
        file_name VARCHAR,
        content BLOB
    )
""")

parquet_file = '~/data/midiset-1M5/midiset-1M5.parquet'
# 导入 Parquet 数据
results = con.execute(f"COPY my_table FROM '{parquet_file}' (FORMAT PARQUET)")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [10]:
con.execute("CREATE UNIQUE INDEX idx_id ON my_table (file_name)")

In [11]:
num = 10000
files = con.execute(f"SELECT file_name FROM my_table ORDER BY random() LIMIT {num}").pl().to_series().to_list()
files[:5]

['128c9f6d57aa45b7c0ac1e776edbcf81.mid',
 '6c7c45338a71c445fdb52a6fffe7d59c.mid',
 'b4343f3d16c4e03e09f3cca7b536660b.mid',
 '8af9e2a316c056aac69d80ae8dae02b7.mid',
 '10c6368676097c871035ac0f0075a995.mid']

In [12]:
result = con.execute("SELECT content FROM my_table WHERE file_name = '0eaf3298fd35247fdb07b175e4933ccb.mid'").pl().to_series()[0]
print(result)

b'MThd\x00\x00\x00\x06\x00\x01\x00\x08\x00`MTrk\x00\x00\x00"\x00\xffY\x02\x00\x00\x00\xffT\x05`\x00\x00\x00\x00\x00\xffQ\x03\x04wJ\x00\xffX\x04\x04\x02\x18\x08\x00\xff/\x00MTrk\x00\x00\tF\x00\xff!\x01\x00\x00\xff\x7f\x07\x00\x00\x00j\x00\x04\x01\x00\xff\x03\rGuitar Melody\x00\xc0\x19\x82 \x90;\x7f0;\x00\x009\x7f09\x00\x00\xc0\x19\x00\x907\x7f07\x00\x006\x7f06\x00\x007\x7f07\x00\x009\x7f09\x00\x007\x7f07\x00\x002\x7f02\x00\x00/\x7f0/\x00\x000\x7f00\x00\x00\xc0\x19\x00\x902\x7f02\x00\x004\x7f04\x00\x002\x7f02\x00\x00/\x7f0/\x00\x002\x7f02\x00\x004\x7f04\x00\x007\x7f07\x00\x009\x7f09\x00\x00\xc0\x19\x00\x90:\x7f0:\x00\x00;\x7f0;\x00\x00>\x7f0>\x00\x00@\x7f0@\x00\x00>\x7f0>\x00\x00;\x7f0;\x00\x007\x7f07\x00\x009\x7f09\x00\x00\xc0\x19\x00\x90;\x7f`;\x00\x009\x7f`9\x00\x009\x7f`9\x00\x00;\x7f0;\x00\x009\x7f09\x00\x00\xc0\x19\x00\x907\x7f07\x00\x006\x7f06\x00\x007\x7f07\x00\x009\x7f09\x00\x007\x7f07\x00\x002\x7f02\x00\x00/\x7f0/\x00\x000\x7f00\x00\x00\xc0\x19\x00\x902\x7f02\x00\x004\x7f04\x00

In [ ]:
print(len(files))

10000


In [16]:
%%timeit
files = con.execute(f"SELECT file_name FROM my_table ORDER BY random() LIMIT {num}").pl().to_series().to_list()

14.4 ms ± 1.22 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit
files = con.execute(f"SELECT file_name FROM my_table ORDER BY random() LIMIT {num}").pl().to_series().to_list()
for file in files:
    result = con.execute(f"SELECT content FROM my_table WHERE file_name = '{file}'").pl().to_series()[0]

3.75 s ± 261 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
# %%timeit

files = con.execute(f"SELECT file_name FROM my_table ORDER BY random() LIMIT {num}").pl().to_series().to_list()
results = con.executemany(f"SELECT content FROM my_table WHERE file_name = ?", files).pl().to_series()

InvalidInputException: Invalid Input Error: Prepared parameters can only be passed as a list or a dictionary

In [14]:
# 批量查找
def batch_find(file_names):
    placeholders = ', '.join(['?'] * len(file_names))
    query = f"SELECT file_name, content FROM my_table WHERE file_name IN ({placeholders})"
    df = con.execute(query, file_names).pl()
    return {
        name: content for name, content in df.iter_rows()
    }

In [15]:
%%timeit
result = batch_find(files)

138 ms ± 23.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
